# Testeo para Spider HTML

In [218]:
import requests
import re
from bs4 import BeautifulSoup
import time

In [219]:
url = 'https://www.amazon.ae/Ariel-Automatic-Laundry-Detergent-Original/dp/B076L3PYX6/ref=pd_vtp_h_pd_vtp_h_d_sccl_2/257-2861550-0109024?pd_rd_w=Gh3Nt&content-id=amzn1.sym.84eb92b9-87b1-451d-8969-226f5a04c3f5&pf_rd_p=84eb92b9-87b1-451d-8969-226f5a04c3f5&pf_rd_r=TWHGYCDGMJFWJDP009YD&pd_rd_wg=Z3w7T&pd_rd_r=094a09ae-684d-4c15-b779-42bfe86a86a4&pd_rd_i=B076L3PYX6&psc=1'

In [220]:
payload = {}
headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
  'Accept-Language': 'es-AR,es;q=0.8,en-US;q=0.5,en;q=0.3',
  'Accept-Encoding': 'gzip, deflate, br',
  'Alt-Used': 'www.amazon.ae',
  'Connection': 'keep-alive',
  'Cookie': 'session-id=257-2861550-0109024; session-id-time=2082787201l; i18n-prefs=AED; csm-hit=tb:66MHB1N659SQMYKBDPHY+s-66MHB1N659SQMYKBDPHY|1709900943401&t:1709900943401&adb:adblk_yes; ubid-acbae=257-3832480-6790808; session-token=yZTuAXblFNNs1RFEQfOxeGfBGyes9xRc3+qGG+iXx0C60BVV80s+sbfy/j65jCK1jQtSl5xwICBmJs7utHp3CaKph2GF5THAcvxPGmo1nxNa50YcGo/+tUTWX+w2AVE6c2+WQ0AegEy1u5NG2zGtk8BPpwvC3hMN4CEh5Hc+9R8OPiQ5Y0tQ9oQ6HUTce3Bq1LZXr2tkHNdCDheNIHc/GquFp2Rn9QJSRNQmfRAfMC88OfF9p0BOBgR7h5cEngWS7KEnnLdmrK9As3nI2xd7VjLn4+33zSQM74y9mcRUN5MEMHsbRu2cDNeb6cZGnrtWX8Myrxg+qh5MWwfPxbWHVasAhpBkz5Ye; lc-acbae=en_AE; ld=AZAESOA_pcdisc_C; s_pers=%20s_fid%3D3504FFD318A2EA58-3D892D2C174744DD%7C1866804897117%3B%20s_dl%3D1%7C1708953897117%3B%20gpv_page%3DAZ%253ASD%253ASOA-home%7C1708953897118%3B%20s_ev15%3D%255B%255B%2527AZAESOA_pcdisc_C%2527%252C%25271708952097118%2527%255D%255D%7C1866804897118%3B; session-token=o8qfeKRQusBex7pBRdlO61Y2/tOAcJNh2M18BQpj/rQxPnp9T9RsTSBY6NjN1/4wwY92p0g+qu5+nvnsJzUliBL9XqUm8LNztzZl/N9qU8u08z95RvwrkDzqfIRyyx3AbF3FfyLA0zyj6uTHcnCyhnnq+ipGZZyw8jwMOhE+XSXCgOclGR8elZesjrhwxTfaHLDk5wg8BzP6ogCKOq1WjMNod1S9DEhJyj8MM9iqVNEfGhxgI93ezfLQd7EooI8udZbFC7flKcZ0q8zLON+3uqJsuJqmGM9x1N+S4VmdzfOEcLYG6UMONdvnOTYVP/YUjxPvB351kJ6gSDCHcccblC1HplUElDRO',
  'Upgrade-Insecure-Requests': '1',
  'Sec-Fetch-Dest': 'document',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-Site': 'cross-site',
  'Sec-Fetch-User': '?1',
  'Sec-GPC': '1',
  'TE': 'trailers'
}

In [221]:
response = requests.request("GET", url, headers=headers, data=payload)
html_content = response.text
soup = BeautifulSoup(html_content, "html.parser")

In [222]:
buy_box = soup.select_one('#merchantInfoFeature_feature_div > div:nth-child(2) > div:nth-child(1) > span:nth-child(1) > a:nth-child(1)').text if soup.select_one('#merchantInfoFeature_feature_div > div:nth-child(2) > div:nth-child(1) > span:nth-child(1) > a:nth-child(1)') != None else ''
if buy_box == '':
    buy_box = soup.select_one('#merchantInfoFeature_feature_div > div:nth-child(2) > div:nth-child(1) > span:nth-child(1)').text if soup.select_one('#merchantInfoFeature_feature_div > div:nth-child(2) > div:nth-child(1) > span:nth-child(1)') != None else ''
if buy_box == '':
    buy_box = soup.select_one('tr.a-spacing-micro:nth-child(2) > td:nth-child(2) > span:nth-child(1) > a:nth-child(1)').text if soup.select_one('tr.a-spacing-micro:nth-child(2) > td:nth-child(2) > span:nth-child(1) > a:nth-child(1)') != None else ''

In [223]:
if buy_box == '' or 'Amazon.ae' not in buy_box:
    # Check if the store is transmed
    others_offer = soup.find_all('span', class_='a-size-small mbcMerchantName')# Catura para testear que lleguen los nombres
    others_offer_container = soup.find_all('div', class_='a-box-inner a-padding-small')  # Assuming class names remain the same

    # Print others_offer for debugging purposes (if needed)
    print(f"OTHERS OFFER:{[o.text.strip() for o in others_offer]}\n")

    for container in others_offer_container:
        title_store = container.find('span', class_='a-size-small mbcMerchantName').text if container.find('span', class_='a-size-small mbcMerchantName') != None else ''
        if title_store and 'Amazon.ae' in title_store.text.strip():
            final_price_element = container.find('span', class_='a-size-medium a-color-price')
            if final_price_element:
                final_price = fc.just_price(price_str=final_price_element.text.strip(), dotted_price=False)
                is_transmed = True
                break  # Exit the loop after finding the first Transmed offer

OTHERS OFFER:[]



In [224]:
others_string = ''
others_offer_container = soup.find_all('div', class_='a-box-inner a-padding-small')
for i in range(1, len(others_offer_container)):
        title_store = others_offer_container[i].find('span', class_='a-size-small mbcMerchantName').text if others_offer_container[i].find('span', class_='a-size-small mbcMerchantName') != None else ''
        store_price = others_offer_container[i].select_one('span.a-size-medium.a-color-price').text if others_offer_container[i].select_one('span.a-size-medium.a-color-price') != None else 0
        others_string += f"Store:{title_store.strip()} Price:{store_price.strip()}, "

In [225]:
print(others_string)

In [226]:
final_price_int = soup.select_one("span.a-price.aok-align-center.reinventPricePriceToPayMargin.priceToPay span span.a-price-whole").text if soup.select_one("span.a-price.aok-align-center.reinventPricePriceToPayMargin.priceToPay span span.a-price-whole") != None else ''
final_price_decimals = soup.select_one("span.a-price.aok-align-center.reinventPricePriceToPayMargin.priceToPay span span.a-price-fraction").text if soup.select_one("span.a-price.aok-align-center.reinventPricePriceToPayMargin.priceToPay span span.a-price-fraction") != None else ''
if final_price_int != None and final_price_int != '':
    final_price_int = re.sub("[^0-9.]", "", final_price_int)
    final_price_int = re.sub("[ ]", "", final_price_int)
    final_price_int = re.sub("[.]", "", final_price_int)

if final_price_decimals != None and final_price_decimals != '':
    final_price_decimals = re.sub("[^0-9.]", "", final_price_decimals)
    final_price_decimals = re.sub("[ ]", "", final_price_decimals)
    final_price_decimals = re.sub("[.]", "", final_price_decimals)

if final_price_int != None and final_price_int != '' and final_price_decimals != None and final_price_decimals != '':
    try:
        final_price = f"{final_price_int}.{final_price_decimals}"
        final_price = float(final_price)
    except:
        final_price = 0
        print(f"1:No se pudo convertir a flotante final_price:{final_price}")

print(f"FinalPrice: {final_price}")

FinalPrice: 99.95


In [227]:
if final_price == 0:
    final_price = soup.select_one("#newAccordionRow_0 > div:nth-child(1) > div:nth-child(1) > h5:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > span:nth-child(1) > span:nth-child(1)").text if soup.select_one("#newAccordionRow_0 > div:nth-child(1) > div:nth-child(1) > h5:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > span:nth-child(1) > span:nth-child(1)") != None else 0 
    final_price = fcs.just_price(price_str=final_price, dotted_price=False)

In [228]:
old_price = soup.select_one(".basisPrice > span:nth-child(1) > span:nth-child(1)").text if soup.select_one(".basisPrice > span:nth-child(1) > span:nth-child(1)") != None else 0
print(f"OldPrice:{old_price}")

OldPrice:0


In [229]:
if old_price == 0 or old_price == '':
    old_price = soup.select_one("td.a-color-secondary:nth-child(2) > span:nth-child(1) > span:nth-child(1)").text if soup.select_one("td.a-color-secondary:nth-child(2) > span:nth-child(1) > span:nth-child(1)") != None else 0


In [230]:
bullet_points_list = soup.select("li.a-spacing-mini")
bullet_points = ''

try:
    for i in range(0, len(bullet_points_list)):
        content = bullet_points_list[i].text.strip()
        if i == len(bullet_points_list) - 1:
            bullet_points += f"•{content}."
        else:
            bullet_points += f"•{content}, "
except:
    print(f"No se pudo extraer los bullets points")
print(f"BulletPoint: {bullet_points}")

BulletPoint: •Number 1 IN STAIN REMOVAL- Stain removal has never been easier.Ariel powder gives your clothes the best stain removal. It removes even the toughest stains, such as yellow underarm stains, providing your clothes with a spotless look., •Keep your clothes fresh- Ariel Powder provides your garments with a fresh clean scent., •Dosage instructions- Use 1 full scoop for the best stain removal., •Full automatic washing machine- Ariel Automatic detergent powder is designed for a full automatic machine, both front load and top load. Ariel detergent powder is a must-have in your laundry room for a constant brilliant clean you can see, feel and smell..


In [231]:
description = soup.find(id='productDescription').text.strip() if soup.find(id='productDescription') != None else ''
print(f"Description:{description}")

Description:Stain removal has never been easier! Ariel detergent powder gives your clothes a stain free clean, no matter what they’ve gone through. Ariel powder removes tough stains in 1 wash, providing your clothes with a spotless look. Protect your garments and make them pop with Ariel detergent powder. Thanks to the outstanding cleaning power of Ariel, our detergent will clean your colours brilliantly, making them look like new! Ariel detergent powder is designed for all types of washing machines, both front load and top load. Ariel detergent powder is a must-have in your laundry room for a constant brilliant clean you can see, feel and smell.


In [234]:
q_images_list = soup.find_all("li", class_="a-spacing-small")
q_images = 0
for i in q_images_list:
    content = i.find('img')["src"] if i.find('img') != None else ''
    if 'https://m.media-amazon.com/' in content and '.jpg' in content:
        q_images += 1
print(q_images - 1)

6


In [236]:
a_plus = soup.find_all('div', class_='celwidget aplus-module module-12 aplus-standard')
print(f"{a_plus}")

[]
